This work is licensed under MIT License

Copyright (c) 2017 Massimiliano Patacchiola

## Introduction
For this week, you'll need to read Chapter 6.3 (pp. 119-128) & Chapter 12  in Reinforcement Learning: An Introduction.

If one had to identify one idea as central and novel to reinforcement learning, it would undoubtedly be temporal-difference (TD) learning. TD learning is a combination of Monte Carlo ideas and dynamic programming (DP) ideas. Like Monte Carlo methods, TD methods can learn directly from raw experience without a model of the environment’s dynamics. Like DP, TD methods update estimates based in part on other learned estimates, without waiting for a final outcome (they bootstrap). The relationship between TD, DP, and Monte Carlo methods is a recurring theme in the theory of reinforcement learning.


**Temporal Difference Learning - a way to incrementaly estimate the return through bootstrapping.**

Tabular TD(0) Algorithm for estimating $v_\pi$

<img src="https://drive.google.com/uc?id=1aSUVhNykd33K55Imy8V17Znqt5Q4beQU"/>


**TD-error:**

$\delta_t = R + \gamma V(S^{'}) - V(S)$


[Open this link](https://www.coursera.org/learn/sample-based-learning-methods/lecture/CEzFc/comparing-td-and-monte-carlo)

TD(0) allows estimating the utility values following a specific policy. We are in the passive learning case for prediction, and we are in model-free reinforcement learning, meaning that we do not have the transition model. To estimate the utility function we can only move in the world. Using again the cleaning robot example I want to show you what does it mean to apply the TD algorithm to a single episode.

Here is the example from previous lab:

In [ ]:
from IPython.display import Image
Image(url='https://drive.google.com/uc?id=1Vjvu4CoUYyvbmKId0MB0tlodqwhjGjWY', width=600)

Let's consider the **Episode 1** in the example above. The robot starts at (1,1) and reaches the terminal state at (4,3) after seven steps.

<img src="https://drive.google.com/uc?id=1ls4d3VWMBjv7yecTu4TGSQr3JPPKvFyg"/>

Applying the TD algorithm means to move step by step considering only the state at t and the state at t+1.

The TD(0) algorithm ignores the past states as shown by the shadow I added above those states. Applying the algorithm to the episode $(\gamma=0.9, \alpha=0.1)$ leads to the following changes in the utility matrix:

<img src="https://drive.google.com/uc?id=1rf2llL7qpnr7ow-lRAdKI-dB2JjS0UxN"/>


- The matrix is initialised with zeros.

- At k=1, The calculation for updating the utility at (1,1) is: 

   `0.0 + 0.1 (-0.04 + 0.9 (0.0) - 0.0) = -0.004`
   
   Similarly to (1,1) the algorithm updates the state at (1,2).

- At k=3 the robot goes back and the calculation take the form:

   `0.0 + 0.1 (-0.04 + 0.9 (-0.004) - 0.0) = -0.00436`

- At k=4 the robot changes again its direction. In this case the algorithm update for the second time the state (1,2) as follow:

   `-0.004 + 0.1 (-0.04 + 0.9 (-0.00436) + 0.004) = -0.0079924`


- The same process is applied until the end of the episode.

In [ ]:
"""Class for creating a gridworld of arbitrary size and with arbitrary obstacles.
Each state of the gridworld should have a reward. The transition matrix is defined
as the probability of executing an action given a command to the robot.
"""

import numpy as np

class GridWorld:

    def __init__(self, tot_row, tot_col):
        self.action_space_size = 4
        self.world_row = tot_row
        self.world_col = tot_col
        self.transition_matrix = np.ones((self.action_space_size, self.action_space_size))/ self.action_space_size
        self.reward_matrix = np.zeros((tot_row, tot_col))
        self.state_matrix = np.zeros((tot_row, tot_col))
        self.position = [np.random.randint(tot_row), np.random.randint(tot_col)]      

    def setTransitionMatrix(self, transition_matrix):
        '''Set the reward matrix.

        The transition matrix here is intended as a matrix which has a line
        for each action and the element of the row are the probabilities to
        executes each action when a command is given. For example:
        [[0.55, 0.25, 0.10, 0.10]
         [0.25, 0.25, 0.25, 0.25]
         [0.30, 0.20, 0.40, 0.10]
         [0.10, 0.20, 0.10, 0.60]]

        This matrix defines the transition rules for all the 4 possible actions.
        The first row corresponds to the probabilities of executing each one of
        the 4 actions when the policy orders to the robot to go UP. In this case
        the transition model says that with a probability of 0.55 the robot will
        go UP, with a probaiblity of 0.25 RIGHT, 0.10 DOWN and 0.10 LEFT.
        '''
        if(transition_matrix.shape != self.transition_matrix.shape):
            raise ValueError('The shape of the two matrices must be the same.') 
        self.transition_matrix = transition_matrix

    def setRewardMatrix(self, reward_matrix):
        '''Set the reward matrix.

        '''
        if(reward_matrix.shape != self.reward_matrix.shape):
            raise ValueError('The shape of the matrix does not match with the shape of the world.')
        self.reward_matrix = reward_matrix

    def setStateMatrix(self, state_matrix):
        '''Set the obstacles in the world.

        The input to the function is a matrix with the
        same size of the world 
        -1 for states which are not walkable.
        +1 for terminal states
         0 for all the walkable states (non terminal)
        The following matrix represents the 4x3 world
        used in the series "dissecting reinforcement learning"
        [[0,  0,  0, +1]
         [0, -1,  0, +1]
         [0,  0,  0,  0]]
        '''
        if(state_matrix.shape != self.state_matrix.shape):
            raise ValueError('The shape of the matrix does not match with the shape of the world.')
        self.state_matrix = state_matrix

    def setPosition(self, index_row=None, index_col=None):
        ''' Set the position of the robot in a specific state.

        '''
        if(index_row is None or index_col is None): self.position = [np.random.randint(tot_row), np.random.randint(tot_col)]
        else: self.position = [index_row, index_col]

    def render(self):
        ''' Print the current world in the terminal.

        O represents the robot position
        - respresent empty states.
        # represents obstacles
        * represents terminal states
        '''
        graph = ""
        for row in range(self.world_row):
            row_string = ""
            for col in range(self.world_col):
                if(self.position == [row, col]): row_string += u" \u25CB " # u" \u25CC "
                else:
                    if(self.state_matrix[row, col] == 0): row_string += ' - '
                    elif(self.state_matrix[row, col] == -1): row_string += ' # '
                    elif(self.state_matrix[row, col] == +1): row_string += ' * '
            row_string += '\n'
            graph += row_string 
        print (graph)            

    def reset(self, exploring_starts=False):
        ''' Set the position of the robot in the bottom left corner.

        It returns the first observation
        '''
        if exploring_starts:
            while(True):
                row = np.random.randint(0, self.world_row)
                col = np.random.randint(0, self.world_col)
                if(self.state_matrix[row, col] == 0): break
            self.position = [row, col]
        else:
            self.position = [self.world_row-1, 0]
        return self.position

    def step(self, action):
        ''' One step in the world.

        [observation, reward, done = env.step(action)]
        The robot moves one step in the world based on the action given.
        The action can be 0=UP, 1=RIGHT, 2=DOWN, 3=LEFT
        @return observation the position of the robot after the step
        @return reward the reward associated with the next state
        @return done True if the state is terminal  
        '''
        if(action >= self.action_space_size): 
            raise ValueError('The action is not included in the action space.')

        #Based on the current action and the probability derived
        #from the trasition model it chooses a new actio to perform
        action = np.random.choice(4, 1, p=self.transition_matrix[int(action),:])

        #Generating a new position based on the current position and action
        if(action == 0): new_position = [self.position[0]-1, self.position[1]]   #UP
        elif(action == 1): new_position = [self.position[0], self.position[1]+1] #RIGHT
        elif(action == 2): new_position = [self.position[0]+1, self.position[1]] #DOWN
        elif(action == 3): new_position = [self.position[0], self.position[1]-1] #LEFT
        else: raise ValueError('The action is not included in the action space.')

        #Check if the new position is a valid position
        #print(self.state_matrix)
        if (new_position[0]>=0 and new_position[0]<self.world_row):
            if(new_position[1]>=0 and new_position[1]<self.world_col):
                if(self.state_matrix[new_position[0], new_position[1]] != -1):
                    self.position = new_position

        reward = self.reward_matrix[self.position[0], self.position[1]]
        #Done is True if the state is a terminal state
        done = bool(self.state_matrix[self.position[0], self.position[1]])
        return self.position, reward, done


In [ ]:
#In this example I will use the class gridworld to generate a 3x4 world
#in which the cleaning robot will move. Using the TD(0) algorithm I
#will estimate the utility values of each state.

import numpy as np

def update_utility(utility_matrix, observation, new_observation, 
                   reward, alpha, gamma):
    '''Return the updated utility matrix

    @param utility_matrix the matrix before the update
    @param observation the state obsrved at t
    @param new_observation the state observed at t+1
    @param reward the reward observed after the action
    @param alpha the ste size (learning rate)
    @param gamma the discount factor
    @return the updated utility matrix
    '''
    u_t = utility_matrix[observation[0], observation[1]]
    u_t1 = utility_matrix[new_observation[0], new_observation[1]]
    utility_matrix[observation[0], observation[1]] += \
        alpha * (reward + gamma * u_t1 - u_t)
    return utility_matrix


The main loop is much simpler than the one used in MC methods. In this case we do not have any first-visit constraint and the only thing to do is to apply the update rule.

In [ ]:
env = GridWorld(3, 4)

#Define the state matrix
state_matrix = np.zeros((3,4))
state_matrix[0, 3] = 1
state_matrix[1, 3] = 1
state_matrix[1, 1] = -1
print("State Matrix:")
print(state_matrix)

#Define the reward matrix
reward_matrix = np.full((3,4), -0.04)
reward_matrix[0, 3] = 1
reward_matrix[1, 3] = -1
print("Reward Matrix:")
print(reward_matrix)

#Define the transition matrix
transition_matrix = np.array([[0.8, 0.1, 0.0, 0.1],
                              [0.1, 0.8, 0.1, 0.0],
                              [0.0, 0.1, 0.8, 0.1],
                              [0.1, 0.0, 0.1, 0.8]])

#Define the policy matrix
#This is the optimal policy for world with reward=-0.04
policy_matrix = np.array([[1,      1,  1,  -1],
                          [0, np.NaN,  0,  -1],
                          [0,      3,  3,   3]])
print("Policy Matrix:")
print(policy_matrix)

env.setStateMatrix(state_matrix)
env.setRewardMatrix(reward_matrix)
env.setTransitionMatrix(transition_matrix)

utility_matrix = np.zeros((3,4))
gamma = 0.999
alpha = 0.1 #constant step size
tot_epoch = 300000
print_epoch = 1000

for epoch in range(tot_epoch):
    #Reset and return the first observation
    observation = env.reset(exploring_starts=False)
    for step in range(1000):
        #Take the action from the action matrix
        action = policy_matrix[observation[0], observation[1]]
        #Move one step in the environment and get obs and reward
        new_observation, reward, done = env.step(action)
        utility_matrix = update_utility(utility_matrix, observation, 
                                        new_observation, reward, alpha, gamma)
        observation = new_observation
        #print(utility_matrix)
        if done: break

    if(epoch % print_epoch == 0):
        print("")
        print("Utility matrix after " + str(epoch+1) + " iterations:") 
        print(utility_matrix)
#Time to check the utility matrix obtained
print("Utility matrix after " + str(tot_epoch) + " iterations:")
print(utility_matrix)

State Matrix:
[[ 0.  0.  0.  1.]
 [ 0. -1.  0.  1.]
 [ 0.  0.  0.  0.]]
Reward Matrix:
[[-0.04 -0.04 -0.04  1.  ]
 [-0.04 -0.04 -0.04 -1.  ]
 [-0.04 -0.04 -0.04 -0.04]]
Policy Matrix:
[[ 1.  1.  1. -1.]
 [ 0. nan  0. -1.]
 [ 0.  3.  3.  3.]]

Utility matrix after 1 iterations:
[[-0.004 -0.004  0.1    0.   ]
 [-0.004  0.     0.     0.   ]
 [-0.004  0.     0.     0.   ]]

Utility matrix after 1001 iterations:
[[0.85596565 0.91591797 0.98249234 0.        ]
 [0.79805483 0.         0.77400683 0.        ]
 [0.73922368 0.69811124 0.         0.        ]]

Utility matrix after 2001 iterations:
[[0.84988321 0.90812521 0.93983996 0.        ]
 [0.79912406 0.         0.66853178 0.        ]
 [0.74985461 0.68738863 0.         0.        ]]

Utility matrix after 3001 iterations:
[[0.81092333 0.84305252 0.88295336 0.        ]
 [0.78544901 0.         0.56353571 0.        ]
 [0.75976653 0.70960479 0.         0.        ]]

Utility matrix after 4001 iterations:
[[0.85100265 0.90445579 0.94408856 0.        ]

### $TD(\lambda)$ and eligibility traces

As I told you in the previous section, the TD(0) algorithm does not take into account past states. What matters in TD(0) is the current state and the state at $t+1$. However, it would be useful to extend what has been learned at  $t+1$ also to previous states, this would accelerate the learning. To achieve this objective it is necessary to have a short-term memory mechanism to store the states that have been visited in the last steps. For each state $s$ at time $t$ we can define $e_t(s)$ as the eligibility trace:

$$e_t(s)= \begin{cases}\gamma \lambda e_{t-1}(s) & \text { if } s \neq s_t \\ \gamma \lambda e_{t-1}(s)+1 & \text { if } s=s_t\end{cases}$$

Here $\gamma$ is the discount rate and $λ ∈[0,1]$ is a decay parameter called trace-decay or accumulating trace defining the update weight for each state visited. When $0<λ<1$ the traces decrease in time, giving a small weight to infrequent states. For the particular case 
$λ=0$ we have $TD(0)$, and only the previous prediction is updated. For λ=1
 we have TD(1) where all the previous predictions are equally updated. TD(1) can be considered an extension of MC methods using a TD framework. In MC methods we need to wait the end of the episode in order to update the states. In TD(1) we can update all the previous states online, we do not need the end of the episode. Let's see now what happens to a specific state trace during an episode. I will take into account an episode with seven visits where five states are visited. The state $s_1$ is visited twice during the episode. Let's see what happens to its trace.

<img src="https://drive.google.com/uc?id=1tx6xkubt-4rB0yj_R-0ucVld7zayw5cp"/>

At the beginning the trace is equal to zero. After the first visit to 
$s_1$
 (second step) the trace goes up to 1 and then it starts decaying. After the second visit (fourth step) +1 is added to the current value (0.25) obtaining a final trace of 1.25. After that point the state 
$s_1$
 is no more visited and the trace slowly goes to zero. How does TD(λ) update the utility function? In TD(0) we saw that a uniform shadow was added in the graphical illustration to represent the inaccessibility of previous states. In TD(λ) the previous states are accessible but they are updated based on the eligibility trace value. States with a small eligibility trace will be updated of a small amount whereas states with high eligibility traces will be substantially updated.

 <img src="https://drive.google.com/uc?id=1kA_jNtRwzADjmwDI1pOUHS02UkNF4Nn9"/>

Graphically we can represent TD(λ) with a non-uniform shadow partially hiding previous states. Now it’s time to define the update rule for TD(λ).

Recall that the estimation error that we defined at the beginning of the lab (above):

$\delta_t = R + \gamma U(S^{'}) - U(S)$

we can update the utility function as follows:

$$U_t(s) = U_t(s) + \alpha \delta_t e_t(s); s \in S$$




To help you understand the differences between TD(0) and TD(λ) I build a 4x3 grid world where the reward is zero for all the states but the two terminal states. The utility matrix is initialised with zeros. The episode I will take into account has five visits, the robot starts at state (1,1) and it arrives at the charging station (4,3) following the optimal path.

<img src="https://drive.google.com/uc?id=1wRoig-j2o9Yzwm1QsaEjzZq3TTfW0cD1"/>

The results of the update for TD(0) and TD(λ) are the same (zero) along all the visit but the last one. When the robot reaches the charging station (reward +1.0) the update rule returns a positive value. In TD(0) the result is propagated only to the previous state (3,3). In TD(λ) the result is propagated back to all previous states thanks to the eligibility trace. The decay value of the trace gives more weight to the last states. As I told you the utility trace mechanism helps to speed up the convergence. It is easy to understand why, if you consider that in our example TD(0) needs five episodes in order to reach the same results of TD(λ).

<img src="https://drive.google.com/uc?id=1wbuJ7_4bd_KPM2rNfYq7vHUcbM1Sm5XJ"/>

In [ ]:
import numpy as np

def update_utility(utility_matrix, trace_matrix, alpha, delta):
    '''Return the updated utility matrix
    @param utility_matrix the matrix before the update
    @param alpha the step size (learning rate)
    @param delta the error (Taget-OldEstimte) 
    @return the updated utility matrix
    '''
    utility_matrix += alpha * delta * trace_matrix
    return utility_matrix

def update_eligibility(trace_matrix, gamma, lambda_):
    '''Return the updated trace_matrix
    @param trace_matrix the eligibility traces matrix
    @param gamma discount factor
    @param lambda_ the decaying value
    @return the updated trace_matrix
    '''
    trace_matrix = trace_matrix * gamma * lambda_
    return trace_matrix


The main loop introduces some new components compared to the TD(0) case. We have the estimation of delta in a separate line and the management of the trace_matrix in two lines. First of all the states are increased (+1) and then they are decayed.

In [ ]:
env = GridWorld(3, 4)

#Define the state matrix
state_matrix = np.zeros((3,4))
state_matrix[0, 3] = 1
state_matrix[1, 3] = 1
state_matrix[1, 1] = -1
print("State Matrix:")
print(state_matrix)

#Define the reward matrix
reward_matrix = np.full((3,4), -0.04)
reward_matrix[0, 3] = 1
reward_matrix[1, 3] = -1
print("Reward Matrix:")
print(reward_matrix)

#Define the transition matrix
transition_matrix = np.array([[0.8, 0.1, 0.0, 0.1],
                              [0.1, 0.8, 0.1, 0.0],
                              [0.0, 0.1, 0.8, 0.1],
                              [0.1, 0.0, 0.1, 0.8]])

#Define the policy matrix
#This is the optimal policy for world with reward=-0.04
policy_matrix = np.array([[1,      1,  1,  -1],
                          [0, np.NaN,  0,  -1],
                          [0,      3,  3,   3]])
print("Policy Matrix:")
print(policy_matrix)

#Define and print the eligibility trace matrix
trace_matrix = np.zeros((3,4))
print("Trace Matrix:")
print(trace_matrix)

env.setStateMatrix(state_matrix)
env.setRewardMatrix(reward_matrix)
env.setTransitionMatrix(transition_matrix)

utility_matrix = np.zeros((3,4))
gamma = 0.999 #discount rate
alpha = 0.1 #constant step size
lambda_ = 0.5 #decaying factor
tot_epoch = 300000
print_epoch = 1000

for epoch in range(tot_epoch):
    #Reset and return the first observation
    observation = env.reset(exploring_starts=True)        
    for step in range(1000):
        #Take the action from the action matrix
        action = policy_matrix[observation[0], observation[1]]
        #Move one step in the environment and get obs and reward
        new_observation, reward, done = env.step(action)
        #Estimate the error delta (Target - OldEstimate)
        delta = reward + gamma * utility_matrix[new_observation[0], new_observation[1]] - \
                                  utility_matrix[observation[0], observation[1]]
        #Adding +1 in the trace matrix for the state visited
        trace_matrix[observation[0], observation[1]] += 1
        #Update the trace matrix (decaying)
        trace_matrix = update_eligibility(trace_matrix, gamma, lambda_)
        #Update the utility matrix
        utility_matrix = update_utility(utility_matrix, trace_matrix, alpha, delta)
        
        observation = new_observation
        if done: break #return

    if(epoch % print_epoch == 0):
        print("")
        print("Utility matrix after " + str(epoch+1) + " iterations:") 
        print(utility_matrix)
        print(trace_matrix)
#Time to check the utility matrix obtained
print("Utility matrix after " + str(tot_epoch) + " iterations:")
print(utility_matrix)


State Matrix:
[[ 0.  0.  0.  1.]
 [ 0. -1.  0.  1.]
 [ 0.  0.  0.  0.]]
Reward Matrix:
[[-0.04 -0.04 -0.04  1.  ]
 [-0.04 -0.04 -0.04 -1.  ]
 [-0.04 -0.04 -0.04 -0.04]]
Policy Matrix:
[[ 1.  1.  1. -1.]
 [ 0. nan  0. -1.]
 [ 0.  3.  3.  3.]]
Trace Matrix:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

Utility matrix after 1 iterations:
[[-0.00060299  0.01238406  0.07305168  0.        ]
 [-0.00229919  0.          0.          0.        ]
 [ 0.          0.          0.          0.        ]]
[[0.03109406 0.18687575 0.74900025 0.        ]
 [0.01553148 0.         0.         0.        ]
 [0.         0.         0.         0.        ]]

Utility matrix after 1001 iterations:
[[0.84154357 0.91161497 0.9748658  0.        ]
 [0.79069715 0.         0.75974996 0.        ]
 [0.74573803 0.70608854 0.67372334 0.39178325]]
[[1.86878793e-01 2.49504004e-01 5.02771838e-01 0.00000000e+00]
 [4.66260161e-02 0.00000000e+00 5.43181318e-04 0.00000000e+00]
 [7.75797640e-03 3.87510921e-03 4.50794764e-05 2.53621231e-

Comparing the final utility matrix with the one obtained without the use of eligibility traces in TD(0) you will notice similar values. One could ask: what's the advantage of using eligibility traces? The eligibility traces version converges faster. This advantage become clear when dealing with sparse reward in a large state space. In this case the eligibility trace mechanism can considerably speeds up the convergence propagating what learnt at t+1 back to the last states visited.